[Dataset](https://www.kaggle.com/competitions/playground-series-s4e6/overview), accuracy metric

[Dataset description](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import nn
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("../code/datasets/train.csv", index_col=0)
target = df["Target"].values
data = df.drop(columns=["Target"])

In [3]:
categorial_cols = ['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification','Nacionality',
       "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', "Gender", 'Scholarship holder']

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler


def preprocess(dataset, encoder, scaler, cols):
    dataset = dataset.copy()
    
    data_cat = dataset[categorial_cols]
    data_nc = dataset.drop(columns=categorial_cols)
    
    data_cat_encoded = encoder.fit_transform(data_cat)
    data_nc_scaled = scaler.fit_transform(data_nc)
    
    data_cat_encoded_df = pd.DataFrame(data_cat_encoded, columns=encoder.get_feature_names_out(cols))
    data_nc_scaled_df = pd.DataFrame(data_nc_scaled, columns=data_nc.columns)
    
    final_data = pd.concat([data_cat_encoded_df, data_nc_scaled_df], axis=1)
    
    return final_data

In [5]:
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
scaler = StandardScaler()

data_prec = preprocess(dataset=data, encoder=one_hot_encoder, scaler=scaler, cols=categorial_cols)
data_prec

,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_2,Application mode_3,Application mode_4,Application mode_5,Application mode_7,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,-0.066933,0.718660,0.505317,-0.135127,-0.158418,-0.449110,0.933176
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,0.504003,-1.445110,-1.735681,-0.135127,-0.158418,-0.449110,0.933176
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,-2.065210,-1.445110,-1.735681,-0.135127,1.763675,-0.663578,-0.372698
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,1.270048,1.074940,1.079288,0.575895,-0.135127,-0.158418,-0.449110,0.933176
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.655484,1.360408,0.718660,0.596330,-0.135127,-1.477502,0.980680,0.178079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,0.218535,0.358031,0.175607,-0.135127,0.896848,-1.092515,0.386841
76514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,0.504003,0.718660,0.766123,-0.135127,-0.799116,-1.449963,-1.349882
76515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.754221,-0.146765,-0.573642,0.218535,0.358031,0.319855,2.028889,-0.799116,-1.449963,-1.349882
76516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.141890,-0.146765,0.040921,-2.065210,-1.445110,-1.735681,-0.135127,-1.477502,0.980680,0.178079


In [6]:
def split_data(data, test_size=0.2, seed=42):
    np.random.seed(seed)
    indices = np.random.permutation(len(data))
    test_split = int(len(data) * test_size)

    train_indices, test_indices = indices[test_split:], indices[:test_split]
    return train_indices, test_indices

In [7]:
batch_size = 128
test_size = 0.2
seed = 42

In [8]:
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset

X = torch.tensor(data_prec.values, dtype=torch.float32)
y = torch.tensor(target_encoded, dtype=torch.int64)

processed_dataset = TensorDataset(X, y)

train_idx, test_idx = split_data(data=X, test_size=test_size, seed=seed)

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(test_idx)

train_loader = DataLoader(
    processed_dataset, batch_size=batch_size, sampler=train_sampler
)
val_loader = DataLoader(processed_dataset, batch_size=batch_size, sampler=val_sampler)

In [9]:
X.shape

torch.Size([76518, 280])

In [10]:
class BasicNet(nn.Module):
    def __init__(self):
        super(BasicNet, self).__init__()

        layer = 512
        self.layers = nn.Sequential(    
            nn.Linear(280, layer),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.BatchNorm1d(layer),
            nn.Linear(layer, layer),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.BatchNorm1d(layer),
            nn.Linear(layer, layer),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.BatchNorm1d(layer),
            nn.Linear(layer, 3),
            nn.ReLU(inplace=True),
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.kaiming_normal_(
                    m.weight, mode="fan_out", nonlinearity="relu"
                )
                torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.layers(x)
        return x

In [29]:
from numpy import equal


def train(
    model,
    optimizer,
    loss_fn,
    train_loader,
    val_loader,
    writer=None,
    epochs=1,
    device="cpu",
    model_path="best.pt",
    scheduler=None,
    tolerance=-1,
    tolerance_delta=1e-4,
):
    # best score for checkpointing
    best = 0.0
    # scaler = GradScaler()

    not_improving = 0
    last_loss = None

    # iterating over epochs
    for epoch in range(epochs):
        # training loop description
        train_loop = tqdm(
            enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"
        )
        model.train()
        train_loss = 0.0
        # iterate over dataset
        for data in train_loop:
            # Write your code here
            # Move data to a device, do forward pass and loss calculation, do backward pass and run optimizer
            data = data[1]
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(inputs)

            loss = loss_fn(output, labels)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

            train_loop.set_postfix({"loss": loss.item()})

            if scheduler:
                scheduler.step()

        # write loss to tensorboard
        if writer:
            writer.add_scalar("Loss/train", train_loss / len(train_loader), epoch)

        # Validation
        correct = 0
        total = 0

        with torch.no_grad():
            model.eval()  # evaluation mode
            val_loop = tqdm(enumerate(val_loader, 0), total=len(val_loader), desc="Val")
            for data in val_loop:
                data = data[1]
                inputs, labels = data[0].to(device), data[1].to(device)
                # Write your code here
                output = model(inputs)
                loss = loss_fn(output, labels).item()

                # Get predictions and compare them with labels
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()
                total += len(labels)

                val_loop.set_postfix({"acc": correct / total})

            val_acc = correct / total

            if val_acc > best:
                torch.onnx.export(
        model,
        inputs.to(device),
        model_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"},
                      "output": {0: "batch_size"}}
    )
                torch.save(optimizer.state_dict(), "opimizer.pt")
                best = correct / total

        if epoch != 0:
            if abs(train_loss - last_loss) < tolerance_delta:
                not_improving += 1
                if not_improving == tolerance:
                    print("Stop due to early reaching tolerance_delta")
                    break
            else:
                not_improving = 0

        last_loss = train_loss

    print(best)

In [27]:
def overfit_single_batch(
    train_loader,
    model,
    loss_fn,
    device="cpu",
    epochs=1,
    batch_size=1,
    lr=1e-2,
):
    data_iter = iter(train_loader)
    single_batch = next(data_iter)
    inputs, labels = single_batch[0].to(device), single_batch[1].to(device)

    single_batch_loader = DataLoader(
        list(zip(inputs, labels)), batch_size=batch_size, shuffle=True
    )

    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        train_loop = tqdm(
            enumerate(single_batch_loader, 0),
            total=len(single_batch_loader),
            desc=f"Epoch {epoch}",
        )
        train_loss = 0.0
        for i, (batch_inputs, batch_labels) in train_loop:
            optimizer.zero_grad()

            output = model(inputs)
            loss = loss_fn(output, labels)

            train_loss += loss.item()

            loss.backward()
            optimizer.step()

            train_loop.set_postfix({"loss": loss.item()})

        print(f"Epoch {epoch} Training Loss: {train_loss / len(single_batch_loader)}")


overfit_single_batch(
    model=BasicNet(),
    loss_fn=nn.CrossEntropyLoss(),
    train_loader=train_loader,
    device="mps",
    epochs=5,
    batch_size=1,
    lr=1e-2,
)

Epoch 0:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 0 Training Loss: 0.3858492630533874


Epoch 1:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1 Training Loss: 0.07519020914332941


Epoch 2:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 2 Training Loss: 0.05882859512348659


Epoch 3:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 3 Training Loss: 0.0496123798948247


Epoch 4:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 4 Training Loss: 0.044214456604095176


In [31]:
device = "mps"

epochs = 1

tolerance = 7
tolerance_delta = 1e-4

In [23]:
import mlflow
import mlflow.pytorch
from mlflow.models import infer_signature

mlflow.set_tracking_uri(uri="http://127.0.0.1:8090")

In [36]:
model = BasicNet()
model.to(device=device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=1)

In [43]:
with mlflow.start_run() as run:
    train(
        model=model,
        optimizer=optimizer,
        loss_fn=loss_function,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=epochs,
        tolerance=tolerance,
        tolerance_delta=tolerance_delta,
    )

    data_iter = iter(train_loader)
    inputs, labels = next(data_iter)

    signature = infer_signature(
        inputs.numpy(), model(inputs.to(device)).detach().cpu().numpy()
    )
    model_info = mlflow.pytorch.log_model(
        pytorch_model=model,
        artifact_path="pytorch_model",
        signature=signature,
        input_example=inputs.numpy(),
        registered_model_name="pytorch_model",
    )

    pytorch_pyfunc = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

    val_iter = iter(val_loader)
    X_val, y_val = next(val_iter)

    raw_predictions = pytorch_pyfunc.predict(X_val.numpy())
    predictions = np.argmax(raw_predictions, axis=1)

    print(predictions)

    eval_data = pd.DataFrame(X_val.numpy())
    eval_data["label"] = y_val.numpy()
    eval_data["predictions"] = predictions
    print(eval_data.shape)

    results = mlflow.evaluate(
        data=eval_data,
        model_type="classifier",
        targets="label",
        predictions="predictions",
        evaluators=["default"],
    )

print(f"metrics:\n{results.metrics}")
print(f"artifacts:\n{results.artifacts}")

Epoch 0:   0%|          | 0/479 [00:00<?, ?it/s]

Val:   0%|          | 0/120 [00:00<?, ?it/s]

0.731947984055414


Registered model 'pytorch_model' already exists. Creating a new version of this model...
2024/09/20 23:17:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: pytorch_model, version 8
Created version '8' of model 'pytorch_model'.


2024/09/20 23:18:16 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.

2024/09/20 23:19:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run nimble-carp-50 at: http://127.0.0.1:8090/#/experiments/0/runs/5ad6645f7eae4afc9ce2b3b651d3104d.
2024/09/20 23:19:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8090/#/experiments/0.


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/5ad6645f7eae4afc9ce2b3b651d3104d/artifacts:
##### File pytorch_model/data/pickle_module_info.txt #####
API request to http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/5ad6645f7eae4afc9ce2b3b651d3104d/artifacts/pytorch_model/data/pickle_module_info.txt failed with exception HTTPConnectionPool(host='127.0.0.1', port=8090): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/0/5ad6645f7eae4afc9ce2b3b651d3104d/artifacts/pytorch_model/data/pickle_module_info.txt (Caused by ResponseError('too many 500 error responses'))
##### File pytorch_model/data/model.pth #####
API request to http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/5ad6645f7eae4afc9ce2b3b651d3104d/artifacts/pytorch_model/data/model.pth failed with exception HTTPConnectionPool(host='127.0.0.1', port=8090): Max retries exceeded with url: /api/2.0

In [38]:
# import mlflow
# import mlflow.onnx
# import numpy as np
# import onnx
# import pandas as pd
# import torch
# from mlflow.models import infer_signature

# with mlflow.start_run() as run:
#     model_path = "model.onnx"


#     train(
#         model=model,
#         optimizer=optimizer,
#         loss_fn=loss_function,
#         train_loader=train_loader,
#         val_loader=val_loader,
#         device=device,
#         epochs=epochs,
#         tolerance=tolerance,
#         tolerance_delta=tolerance_delta,
#         model_path=model_path
#     )

#     data_iter = iter(train_loader)
#     inputs, labels = next(data_iter)

#     onnx_model = onnx.load(model_path)
#     signature = infer_signature(inputs.numpy(), model(inputs.to(device)).detach().cpu().numpy())
#     model_info = mlflow.onnx.log_model(
#         onnx_model=onnx_model,
#         artifact_path="onnx_model",
#         signature=signature,
#         input_example=inputs.numpy(),
#         registered_model_name="onnx_model"
#     )

#     onnx_pyfunc = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

#     val_iter = iter(val_loader)
#     X_val, y_val = next(val_iter)

#     raw_predictions = onnx_pyfunc.predict(X_val.numpy())
#     predictions = np.argmax(raw_predictions, axis=1)

#     print(predictions)

#     eval_data = pd.DataFrame(X_val.numpy())
#     eval_data["label"] = y_val.numpy()
#     eval_data["predictions"] = predictions
#     print(eval_data.shape)

#     results = mlflow.evaluate(
#         data=eval_data,
#         model_type="classifier",
#         targets="label",
#         predictions="predictions",
#         evaluators=["default"],
#     )

# print(f"metrics:\n{results.metrics}")
# print(f"artifacts:\n{results.artifacts}")

Epoch 0:   0%|          | 0/479 [00:00<?, ?it/s]

Val:   0%|          | 0/120 [00:00<?, ?it/s]

0.6687577599163562


Successfully registered model 'onnx_model'.
2024/09/20 23:09:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: onnx_model, version 1
Created version '1' of model 'onnx_model'.


2024/09/20 23:10:51 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.

2024/09/20 23:11:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run masked-colt-777 at: http://127.0.0.1:8090/#/experiments/0/runs/aec42f043144462a98ef8092f3029915.
2024/09/20 23:11:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8090/#/experiments/0.


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/aec42f043144462a98ef8092f3029915/artifacts:
##### File onnx_model/requirements.txt #####
API request to http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/aec42f043144462a98ef8092f3029915/artifacts/onnx_model/requirements.txt failed with exception HTTPConnectionPool(host='127.0.0.1', port=8090): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/0/aec42f043144462a98ef8092f3029915/artifacts/onnx_model/requirements.txt (Caused by ResponseError('too many 500 error responses'))
##### File onnx_model/python_env.yaml #####
API request to http://127.0.0.1:8090/api/2.0/mlflow-artifacts/artifacts/0/aec42f043144462a98ef8092f3029915/artifacts/onnx_model/python_env.yaml failed with exception HTTPConnectionPool(host='127.0.0.1', port=8090): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/0/aec42f043144462a